### Initialization

In [2]:
import pandas as pd
import numpy as np
import os
import webbrowser
import plotly.graph_objects as go

In [3]:
# Loads the updated dataframe (after having used "Data load.ipynb")
df = pd.read_excel("Data/data_updated.xlsx")

In [4]:
# Adds a zero to the course codes that start with zero, since the initial zero is removed during the xlsx save "data_updated.xlsx"
df["Course code"] = df["Course code"].astype(str)
df["Course code"] = df["Course code"].str.zfill(5)

### Course categorization

#### Update info
Courses need to be assigned a SINGLE category in the cell below by inserting their course code. Check the next "Update info" for courses lacking to be assigned a category.

*A LLM is a useful but deceitful helper, so use it with caution and check manually too.*

In [7]:
# Courses are assigned a category here by inserting their course code
circularity = ["28870", "12139", "12132", "23552", "12104", "12143", "42014", "12205", "47202", "41051", "38001", "12111", "12130", "12605", "30755", "41342", "12773", "12953", "12211"]
system = ["42180", "02431", "47203", "02411", "42417", "41938", "41740", "41525", "02003", "42402", "42587", "42879", "42101"]
smart = ["02393", "34052", "34338", "41028", "46760", "34373", "34346", "46797", "46795", "34315", "34755", "34723"]
health = ["27002", "22281", "22284", "27510", "22449", "22475", "12701"]
ai = ["10316", "02160", "02170", "02455", "02450", "02502", "02561", "02492", "02105", "02162", "02805", "42115"]
business = ["23531", "42576", "42380", "38301", "42577", "38106", "38109", "42009", "25352", "38302", "42016", "38104", "10750", "42421"]
manufacturing = ["41743", "41514", "41615", "41501", "41669", "41522", "41511", "34540", "28213", "41653", "41733", "41659", "41512", "41347", "41612", "28244", "41737", "46420", "41749", "41528", "41735", "41739", "41656", "10862", "41822"]
adjacent = ["30530", "10420", "41031", "02441", "23532", "10603", "42543", "10610", "88383", "42893", "10605", "01035", "02402", "02403"]

In [8]:
# Assigns the categories to df
category_lists = {
    "Design for Circularity and Sustainability": list(map(str, circularity)),
    "Systems Engineering": list(map(str, system)),
    "Smart Products": list(map(str, smart)),
    "Health Technology": list(map(str, health)),
    "Artificial Intelligence and Programming": list(map(str, ai)),
    "Business and Entrepreneurship": list(map(str, business)),
    "Manufacturing and Mechanics": list(map(str, manufacturing)),
    "Design Adjacent": list(map(str, adjacent))
}

def find_categories(course):
    return [name for name, codes in category_lists.items() if course in codes]

df['Category'] = df['Course code'].apply(find_categories)
df['Category'] = df['Category'].apply(lambda x: ', '.join(x)) # Is used to identify multi-categorization errors (see the cells below)

#### Update info
The two cells below indicate possible errors for course categorization. Check those cells and adapt the category list above.

In [10]:
#Shows courses that haven't been assigned a category (a course needs ONE category)
df[df["Category"] == ""][["Course code", "URL"]]

,Course code,URL


In [11]:
# Shows courses that have been assigned multiple categories (they may only have one)
df[df["Category"].str.contains(",", na=False)]

,Course code,Course name,Registered students,Exam pass percentage,Average grade,ECTS,Schedule,Level,URL,Workload,Evaluation score,DI percentage,Category


In [12]:
# Creating the different dataframes for the dropdown menu
df_fall = df[df["Schedule"] == "Autumn 13-week"]
df_spring = df[df["Schedule"] == "Spring 13-week"]
df_jan = df[df["Schedule"] == "January 3-week"]
df_jun = df[df["Schedule"] == "June 3-week"]
df_jul = df[df["Schedule"] == "July 3-week"]
df_aug  = df[df["Schedule"] == "August 3-week"]

In [13]:
# Appends schedule values for multi schedule courses

# First, group and merge the "Schedule" column
schedule_combined = df.groupby("Course code", as_index=False).agg({"Schedule": lambda x: ", ".join(x.astype(str).unique())})

# Append the schedule values for the seasonal dataframes
dfs = [df_fall, df_spring, df_jan, df_jun, df_jul, df_aug]
df_names = ["df_fall", "df_spring", "df_jan", "df_jun", "df_jul", "df_aug"]
for i, d in enumerate(dfs):
    dfs[i] = pd.merge(d.drop(columns="Schedule"), schedule_combined, on="Course code", how="left")
df_fall, df_spring, df_jan, df_jun, df_jul, df_aug = dfs

# Then, drop duplicates based on "Course code" in the original df (to retain one row per course code), and keeps the values from the first row
df_dedup = df.drop_duplicates(subset="Course code", keep="first")

# Merge the combined schedule back into the deduplicated df
df_all = pd.merge(df_dedup.drop(columns="Schedule"), schedule_combined, on="Course code", how="left")

In [14]:
# Shows the amount of course types
print("Course count by dataframe")
print(f"All = {len(df)}")
print(f"All (consolidating double-scheduled courses) = {len(df_all)}")
print(f"Autumn 13-week = {len(df_fall)}")
print(f"Spring 13-week = {len(df_spring)}")
print(f"January 3-week = {len(df_jan)}")
print(f"June 3-week = {len(df_jun)}")
print(f"July 3-week = {len(df_jul)}")
print(f"August 3-week = {len(df_aug)}")

Course count by dataframe
All = 119
All (consolidating double-scheduled courses) = 109
Autumn 13-week = 52
Spring 13-week = 43
January 3-week = 9
June 3-week = 11
July 3-week = 0
August 3-week = 4


In [15]:
# Dataframe dictionary
dataframes = {
    "All": df_all,
    "Autumn 13-week": df_fall,
    "Spring 13-week": df_spring,
    "January 3-week": df_jan,
    "June 3-week": df_jun,
    "July 3-week": df_jul,
    "August 3-week": df_aug
}

# Remove empty DataFrames
dataframes = {k: v for k, v in dataframes.items() if not v.empty}

#Category list for the plot
categories = sorted(df_all["Category"].dropna().unique().tolist())

### Plotly and JS

In [17]:
# Centralized styleblock to simplify the callbacks
style_block = """

<style>
    .title-info {
        font-family: Arial, sans-serif;
        font-size: 18px;
    }
    .section-title {
        font-family: Arial, sans-serif;
        font-size: 16px;
        font-weight: bold;
    }
    .course-table {
        font-family: Arial, sans-serif;
        font-size: 14px;
        margin-top: 6px;
        border-collapse: collapse;
        border: 1px solid #ccc; /* adds border around the whole table */
        border-left: 4px solid grey;
        margin-right: 20px;
        display: inline-table;
    }
    
    .course-table td,
    .course-table th {
        font-family: Arial, sans-serif;
        font-size: 14px;
        padding: 2px 8px;
        vertical-align: top;
        border: 1px solid #ccc; /* adds visible lines between cells */
    }

    .course-table td:first-child {
    background-color: #f0f0f0;
    }

    #plot-wrapper {
    width: 1200px;      /* fixed width, matches figure width */
    margin: 0 auto;     /* centers wrapper horizontally */
    }
    
    #plot-container {
        position: relative;
        width: 100%;        /* takes full width of wrapper */
    }
    
    #course-info {
        margin-left: 80px;
        width: 760px;
        background: #fafafa;
        border: 1px solid #ccc;
        border-radius: 8px;
        padding: 12px;
    }

    #reset-button {
        position: absolute;
        top: 144px;
        right: 10px;
        background-color: white;
        border: 1.5px solid #bfc9dc;
        border-radius: 4px;
        padding: 4px 18px 9px 10px;
        font-size: 14px;
        font-family: Arial, sans-serif;
        color: black;
        cursor: pointer;
        z-index: 1;
    }

    #reset-button:hover {
        background-color: #F4FAFF;
    }


</style>
"""

In [18]:
# Reset button
reset_button = """
<button id="reset-button" onclick="location.reload()">🔄 Clear all</button>
"""

In [19]:
# Callback that activates every time a datapoint on the plot is pressed,
# and adds custom UI boxes around legend items.
js_callback = """
<script>
document.addEventListener("DOMContentLoaded", function () {
    // Get the first Plotly graph div on the page
    const graphDiv = document.getElementsByClassName('plotly-graph-div')[0];
    
    // Store the previously selected point to clear it when a new point is clicked
    let previous = null;

    // Function to add a white rectangular box behind each legend item
    function addLegendBoxes() {
        const legendItems = document.querySelectorAll('.legend .traces'); // Select all legend trace items

        legendItems.forEach(item => {
            // Remove old box if it exists to prevent duplicates
            const oldBox = item.querySelector('rect.legend-box');
            if (oldBox) oldBox.remove();

            // Use requestAnimationFrame to ensure SVG elements are ready before measuring
            requestAnimationFrame(() => {
                try {
                    const bbox = item.getBBox(); // Get bounding box of legend item
                    if (bbox.height === 0) return; // Skip if element not visible

                    const svgNS = "http://www.w3.org/2000/svg"; // SVG namespace
                    const rect = document.createElementNS(svgNS, 'rect'); // Create a new rect
                    rect.setAttribute('class', 'legend-box'); // Add class for styling
                    rect.setAttribute('x', bbox.x + 4); // Slight offset for padding
                    rect.setAttribute('y', bbox.y + 1);
                    rect.setAttribute('width', bbox.width);
                    rect.setAttribute('height', bbox.height);
                    rect.setAttribute('fill', 'white'); // Default fill
                    rect.setAttribute('stroke', 'none'); // No border initially
                    rect.setAttribute('stroke-width', '1');
                    rect.setAttribute('rx', '1'); // Rounded corners
                    rect.setAttribute('ry', '1');

                    // Insert the rectangle behind the text of the legend item
                    item.insertBefore(rect, item.firstChild);

                    // Change rectangle style on hover
                    item.addEventListener('mouseenter', () => {
                        rect.setAttribute('fill', '#F4FAFF');
                        rect.setAttribute('stroke', '#bfc9dc');
                    });
                    item.addEventListener('mouseleave', () => {
                        rect.setAttribute('fill', 'white');
                        rect.setAttribute('stroke', 'none');
                    });
                } catch (err) {
                    // Fail silently if getBBox fails (element not ready yet)
                }
            });
        });
    }

    // Function to highlight a clicked data point
    function highlightPoint(data) {
        data.event.stopPropagation(); // Prevent event bubbling

        const point = data.points[0]; // Get the first clicked point
        const curveIndex = point.curveNumber; // Index of the trace
        const pointIndex = point.pointIndex; // Index of the point in the trace

        // Clear previous selection
        if (previous !== null) {
            Plotly.restyle(graphDiv, {'selectedpoints': [null]}, [previous.curveIndex]);
        }

        // Set new selection on the clicked point
        Plotly.restyle(graphDiv, {'selectedpoints': [[pointIndex]]}, [curveIndex]);
        previous = {curveIndex, pointIndex}; // Store new selection

        const cd = point.customdata; // Access custom data attached to the point

        // Build HTML content for the course info panel
        const html = `
            <div class="title-info">
                <h4 style="margin: 0 0 4px 0;">${cd[1]}</h4> <!-- Course name -->
                <hr>
                <table>
                    <tr>
                        <td>
                            <div class="section-title">General course information</div>
                            <table class="course-table">
                                <tr><td><b>Course page</b></td><td><a href="${cd[8]}" target="_blank">${cd[8]}</a></td></tr>
                                <tr><td><b>Course code</b></td><td>${cd[0]}</td></tr>
                                <tr><td><b>Category</b></td><td>${cd[11]}</td></tr>
                                <tr><td><b>Semester</b></td><td>${cd[6]}</td></tr>
                                <tr><td><b>ECTS</b></td><td>${cd[5]} points</td></tr>
                                <tr><td><b>Level</b></td><td>${cd[7]}</td></tr>
                            </table>
                        </td>
                        <td>
                            <div class="section-title">Statistics</div>
                            <table class="course-table">
                                <tr><td><b>Evaluation score</b></td><td>${cd[9]} %</td></tr>
                                <tr><td><b>D&I MSc. students</b></td><td>${cd[10]} %</td></tr>
                                <tr><td><b>Average grade</b></td><td>${cd[4]}</td></tr>
                                <tr><td><b>Exam pass rate</b></td><td>${cd[3]} %</td></tr>
                                <tr><td><b>Course size</b></td><td>${cd[2]} students</td></tr>
                                <tr><td><b>Workload</b></td><td>${cd[12]} hours per week</td></tr>
                            </table>
                        </td>
                    </tr>
                </table>
            </div>
        `;

        // Insert HTML into the course info div
        document.getElementById("course-info").innerHTML = html;
    }

    // Attach the highlightPoint function to plotly_click events
    graphDiv.on('plotly_click', highlightPoint);

    // Add legend box styling on page load and after any plot update
    setTimeout(addLegendBoxes, 50);
    graphDiv.on('plotly_afterplot', () => {
        setTimeout(addLegendBoxes, 50);
    });
});
</script>
"""

In [20]:
# Jitter function to tell data points a part
def add_jitter(series, jitter_fraction=0.01, log_scale=False):
    if log_scale:
        return series * (1 + (np.random.rand(len(series)) - 0.5) * jitter_fraction * 2)
    else:
        scale = (series.max() - series.min()) * jitter_fraction
        return series + (np.random.rand(len(series)) - 0.5) * scale

In [21]:
def build_figure(selected_df_name):
    # Mapping of column names to labels shown in dropdown menus (without units)
    axis_dropdown_labels = {
        "DI percentage": "D&I MSc. rate",
        "Evaluation score": "Evaluation score",
        "Average grade": "Average grade",
        "Exam pass percentage": "Exam pass rate",
        "Registered students": "Course size",  # Display override
        "Workload": "Workload"
    }

    # Mapping of columns to units for axis titles
    axis_units = {
        "DI percentage": "% of Design & Innovation MSc. students per course [log₁₀]",
        "Evaluation score": "% of max score",
        "Average grade": "7-step scale",
        "Exam pass percentage": "% of students signed up for the exam",
        "Registered students": "Students registered for the 1st exam attempt [log₁₀]",
        "Workload": "Hours per week per 5 ECTS"
    }

    # Helper function to format axis title with label and units
    def axis_title(col):
        unit = axis_units[col]
        if unit:
            return f"{axis_dropdown_labels[col]}<br><span style='font-size:12px; color:gray;'>{unit}</span>"
        return axis_dropdown_labels[col]

    # Default axes
    default_x = "DI percentage"
    default_y = "Evaluation score"

    # Create an empty Plotly figure
    fig = go.Figure()
    n_categories = len(categories)

    # Add scatter traces for each dataset and category
    for df_label, df in dataframes.items():
        for cat in categories:
            # Filter the dataframe by category
            df_cat = df[df["Category"] == cat]
            
            # Add a scatter trace for this category
            fig.add_trace(go.Scatter(
                x=df_cat[default_x],
                y=df_cat[default_y],
                mode="markers",
                name=cat,
                visible=(df_label == selected_df_name),  # Only show traces from selected dataset
                marker=dict(size=10, opacity=1),
                selected=dict(marker=dict(size=20, opacity=0.7)),  # Style when selected
                unselected=dict(marker=dict(opacity=1)),           # Style when unselected
                # Store additional data for tooltips and callbacks
                customdata=df_cat[[
                    "Course code", "Course name", "Registered students", "Exam pass percentage",
                    "Average grade", "ECTS", "Schedule", "Level", "URL",
                    "Evaluation score", "DI percentage", "Category", "Workload"
                ]],
                hovertemplate="<b>%{customdata[1]}</b><br>%{customdata[11]}<extra></extra>"  # Tooltip content
            ))

    # =======================
    # Dropdown 1: Dataset selector
    # =======================
    dataset_buttons = []
    for i, df_label in enumerate(dataframes.keys()):
        # Create visibility list for all traces
        vis = [False] * (n_categories * len(dataframes))
        start = i * n_categories
        vis[start:start + n_categories] = [True] * n_categories  # Show only selected dataset
        dataset_buttons.append(dict(
            label=df_label,
            method="update",
            args=[{"visible": vis}]
        ))
    
    # =======================
    # Axis mappings with tick settings and log info
    # =======================
    axis_mappings = {
        "Average grade": {
            "numeric_col": "Average grade numeric",  # Processed numeric version of grades
            "tickvals": [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
            "ticktext": ["02", "3", "4", "5", "6", "7", "8", "9", "10","11", "12", "Pass/<br>Non-Pass"],
            "log": False
        },
        "DI percentage": {
            "numeric_col": "DI percentage",
            "tickvals": [1, 2, 3, 4, 5, 10, 20, 30, 40, 50, 100],
            "ticktext": ["1", "2", "3", "4", "5", "10", "20", "30", "40", "50", "100"],
            "log": True
        },
        "Registered students": {
            "numeric_col": "Registered students",
            "tickvals": [1, 5, 10, 20, 50, 100, 200, 500, 1000],
            "ticktext": ["1", "5", "10", "20", "50", "100", "200", "500", "1000"],
            "log": True
        }
    }

    # =======================
    # Preprocess numeric columns for average grade
    # =======================
    for df_label, df in dataframes.items():
        df["Average grade numeric"] = df["Average grade"].apply(
            lambda x: 13.0 if x == "Pass/Non-Pass" else float(x)
        )

    # =======================
    # Dropdown 2: X-axis selector
    # =======================
    x_buttons = []
    for col in [default_x] + [c for c in axis_dropdown_labels if c != default_x]:
        plot_col = col
        tickvals = None
        ticktext = None
        log_type = "linear"  # Default axis type

        # Use numeric version and ticks if column is in axis_mappings
        if col in axis_mappings:
            plot_col = axis_mappings[col]["numeric_col"]
            tickvals = axis_mappings[col]["tickvals"]
            ticktext = axis_mappings[col]["ticktext"]
            if axis_mappings[col]["log"]:
                log_type = "log"

        # Create the dropdown button for this column
        x_buttons.append(dict(
            label=axis_dropdown_labels[col],
            method="update",
            args=[
                # Update X values with optional jitter
                {"x": [
                    add_jitter(
                        dataframes[df_label][dataframes[df_label]["Category"] == cat][plot_col],
                        jitter_fraction=0.01,
                        log_scale=(axis_mappings.get(col, {}).get("log", False))
                    )
                    for df_label in dataframes.keys() for cat in categories
                ]},
                {
                    "xaxis.title.text": axis_title(col),
                    "xaxis.type": log_type,
                    **({"xaxis.tickmode": "array", "xaxis.tickvals": tickvals, "xaxis.ticktext": ticktext}
                       if tickvals is not None else {"xaxis.tickmode": "auto"})
                }
            ]
        ))
    
    # =======================
    # Dropdown 3: Y-axis selector
    # =======================
    y_buttons = []
    for col in [default_y] + [c for c in axis_dropdown_labels if c != default_y]:
        plot_col = col
        tickvals = None
        ticktext = None
        log_type = "linear"  # Default axis type

        if col in axis_mappings:
            plot_col = axis_mappings[col]["numeric_col"]
            tickvals = axis_mappings[col]["tickvals"]
            ticktext = axis_mappings[col]["ticktext"]
            if axis_mappings[col]["log"]:
                log_type = "log"

        # Create the dropdown button for this column
        y_buttons.append(dict(
            label=axis_dropdown_labels[col],
            method="update",
            args=[
                # Update Y values with optional jitter
                {"y": [
                    add_jitter(
                        dataframes[df_label][dataframes[df_label]["Category"] == cat][plot_col],
                        jitter_fraction=0.01,
                        log_scale=(axis_mappings.get(col, {}).get("log", False))
                    )
                    for df_label in dataframes.keys() for cat in categories
                ]},
                {
                    "yaxis.title.text": axis_title(col),
                    "yaxis.type": log_type,
                    **({"yaxis.tickmode": "array", "yaxis.tickvals": tickvals, "yaxis.ticktext": ticktext}
                       if tickvals is not None else {"yaxis.tickmode": "auto"})
                }
            ]
        ))

    # =======================
    # Layout configuration and dropdown menus
    # =======================
    fig.update_layout(
        updatemenus=[
            dict(buttons=dataset_buttons, x=1.02, y=0.95, xanchor="left", yanchor="top",
                 font=dict(family="Arial", size=14, color="black")),
            dict(buttons=x_buttons, x=1.02, y=0.35, xanchor="left", yanchor="top",
                 font=dict(family="Arial", size=14, color="black")),
            dict(buttons=y_buttons, x=1.23, y=0.35, xanchor="left", yanchor="top",
                 font=dict(family="Arial", size=14, color="black"))
        ],
        title=dict(
            text="Elective Course Selector",
            font=dict(family="Arial", size=24, color="black"),
            xref="paper", x=0, xanchor="left"
        ),
        # Default X-axis settings
        xaxis=dict(
            type="log",
            tickmode='array',
            tickvals=axis_mappings[default_x]["tickvals"],
            ticktext=axis_mappings[default_x]["ticktext"],
            tickfont=dict(family="Arial", size=12, color="gray"),
            ticks="outside", ticklen=0,
            title=dict(text=axis_title(default_x), font=dict(family="Arial", size=18, color="black")),
            automargin=False
        ),
        # Default Y-axis settings
        yaxis=dict(
            tickfont=dict(family="Arial", size=12, color="gray"),
            ticks="outside", ticklen=0,
            title=dict(text=axis_title(default_y), font=dict(family="Arial", size=18, color="black")),
            automargin=False
        ),
        legend_title=dict(text="Learning categories", font=dict(family="Arial", size=18, color="black")),
        legend=dict(xanchor='left', yanchor='top', x=1.02, y=0.85,
                    font=dict(family='Arial', size=14, color="black")),
        dragmode="pan",
        height=680,
        width=1200,
        margin=dict(l=80, r=300, t=120, b=80)
    )

    # =======================
    # Annotations and extra text
    # =======================
    fig.add_annotation(
        text="Elective courses passed by MSc. Design & Innovation students since 2020",
        xref="paper", yref="paper",
        xanchor="left", yanchor="bottom", x=0, y=1.05,
        showarrow=False, font=dict(family="Arial", size=16, color="gray"), align="left"
    )
    fig.add_annotation(
        text='<a href="https://github.com/lindbaek/Elective-Course-Selector" target="_blank">GitHub repository</a>',
        xref="paper", yref="paper",
        xanchor="left", yanchor="bottom", x=0, y=1.01,
        showarrow=False, font=dict(family="Arial", size=12, color="gray"), align="left"
    )
    fig.add_annotation(
        text="Semester placement",
        xref="paper", yref="paper",
        xanchor="left", yanchor="top", x=1.02, y=1,
        showarrow=False, font=dict(family="Arial", size=18, color="black"), align="left"
    )
    fig.add_annotation(
        text=("Tooltip ℹ️<br>"
              "<span style='font-size:12px;'>"
              "• Inspect courses - Hover and click data points<br>"
              "• Filter legend - Double-click a legend item to isolate it<br>"
              "• Reset zoom/drag - Double-click the plot background"
              "</span>"),
        xref="paper", yref="paper",
        xanchor="left", yanchor="bottom", x=1.02, y=0,
        showarrow=False, font=dict(family="Arial", size=16, color="grey"), align="left"
    )

    # Add decorative line and axis labels
    fig.add_shape(
        type="line",
        xref="paper", yref="paper",
        x0=1.02, x1=1.41,
        y0=0.42, y1=0.42,
        line=dict(color="grey", width=1)
    )
    fig.add_annotation(
        text="X-axis",
        xref="paper", yref="paper",
        xanchor="left", yanchor="top", x=1.02, y=0.4,
        showarrow=False, font=dict(family="Arial", size=18, color="black"), align="left"
    )
    fig.add_annotation(
        text="Y-axis",
        xref="paper", yref="paper",
        xanchor="left", yanchor="top", x=1.23, y=0.4,
        showarrow=False, font=dict(family="Arial", size=18, color="black"), align="left"
    )

    return fig

# Build the figure with "All" dataset selected
fig = build_figure("All")

# Convert the figure to HTML without full HTML boilerplate
html_str = fig.to_html(
    full_html=False,
    include_plotlyjs="cd",
    config={
        "scrollZoom": True,
        "displayModeBar": False,
        "displaylogo": False
    }
)

# Full HTML page
full_html = f"""
<html>
<head>
    <title>Elective Course Selector</title>
    {style_block}
</head>
<body>
    <div id="plot-wrapper">
        <div id="plot-container">
            {reset_button}
            {html_str}
        </div>
        <div id="course-info">
            <p style="color:black; font-family:Arial, sans-serif; font-size:18px; font-weight:bold;">
                Click a data point to inspect courses
            </p>
        </div>
    </div>
    {js_callback}
</body>
</html>
"""


# Save and open HTML
with open("index.html", "w", encoding="utf-8") as f:
    f.write(full_html)

webbrowser.open(f"file://{os.path.abspath('index.html')}")

True